In [1]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth  # максимальная глубина дерева
        self.min_samples_split = min_samples_split  # минимальное количество образцов для разбиения
        self.tree = None  # здесь будет храниться построенное дерево

    def fit(self, X, y):
        # cтроим дерево по обучающим данным
        self.tree = self._build_tree(X, y)

    def predict(self, X):
        # предсказываем метки (y) для новых данных
        return np.array([self._predict_sample(sample, self.tree) for sample in X])

    def _gini(self, y):
        # считаем индекс Джини — меру неоднородности (насколько однородны данные)
        classes, counts = np.unique(y, return_counts=True)
        probs = counts / counts.sum()
        return 1 - np.sum(probs ** 2)

    def _best_split(self, X, y):
        # находим наилучшее разбиение по всем признакам
        best_gini = float('inf')
        best_index, best_threshold = None, None

        for feature_index in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask

                if len(y[left_mask]) < self.min_samples_split or len(y[right_mask]) < self.min_samples_split:
                    continue

                gini_left = self._gini(y[left_mask])
                gini_right = self._gini(y[right_mask])
                weighted_gini = (len(y[left_mask]) * gini_left + len(y[right_mask]) * gini_right) / len(y)

                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_index = feature_index
                    best_threshold = threshold

        return best_index, best_threshold

    def _build_tree(self, X, y, depth=0):
        # рекурсивно строим дерево
        num_samples, num_features = X.shape
        num_labels = len(np.unique(y))

        # условия остановки
        if (depth >= self.max_depth or num_labels == 1 or num_samples < self.min_samples_split): # достигнута максимальная глубина
            leaf_value = self._most_common_label(y)
            return {'leaf': True, 'value': leaf_value}

        feature_index, threshold = self._best_split(X, y) # все объекты в листе с одинаковой меткой
        if feature_index is None:
            return {'leaf': True, 'value': self._most_common_label(y)}

        left_mask = X[:, feature_index] <= threshold # в узле слишком мало объектов для разбиения
        right_mask = ~left_mask

        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {
            'leaf': False,
            'feature_index': feature_index,
            'threshold': threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    def _predict_sample(self, sample, tree):
        # предсказание для одного примера
        if tree['leaf']:
            return tree['value']

        if sample[tree['feature_index']] <= tree['threshold']:
            return self._predict_sample(sample, tree['left'])
        else:
            return self._predict_sample(sample, tree['right'])

    def _most_common_label(self, y):
        # наиболее частая метка в листе - наше предсказание
        values, counts = np.unique(y, return_counts=True)
        return values[np.argmax(counts)]

In [2]:
# тест-драйв
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. Генерируем простой датасет
X, y = make_classification(n_samples=100, n_features=2, n_informative=2,
                           n_redundant=0, n_classes=2, random_state=42)

# 2. Делим на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. Обучаем наше дерево
tree = DecisionTree(max_depth=3)
tree.fit(X_train, y_train)

# 4. Предсказываем на тесте
y_pred = tree.predict(X_test)

# 5. Оцениваем точность
acc = accuracy_score(y_test, y_pred)
print(f"Точность дерева решений: {acc:.2f}")


Точность дерева решений: 0.97


In [4]:
# RANDOM FOREST
import numpy as np
from collections import Counter

class RandomForest:
    def __init__(self, n_estimators=10, max_depth=5, min_samples_split=2, max_features=None):
        self.n_estimators = n_estimators  # количество деревьев
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features  # сколько признаков использовать в каждом дереве
        self.trees = []  # список деревьев

    def fit(self, X, y):
        # обучаем лес: каждое дерево на случайной подвыборке
        self.trees = []
        n_samples, n_features = X.shape
        self.max_features = self.max_features or int(np.sqrt(n_features))

        for _ in range(self.n_estimators):
            # случайная выборка
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_sample = X[indices]
            y_sample = y[indices]

            # случайный набор признаков
            feature_indices = np.random.choice(n_features, self.max_features, replace=False)
            X_sample_subset = X_sample[:, feature_indices]

            # то есть у нас каждое дерево обучается на ограниченном кол-ве признаков и строк исходной матрицы

            # обучаем дерево
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_sample_subset, y_sample)

            # сохраняем дерево и признаки, на которых оно обучалось
            self.trees.append((tree, feature_indices))

    def predict(self, X):
        # предсказание: голосование деревьев
        tree_preds = []

        for tree, feature_indices in self.trees:
            X_subset = X[:, feature_indices]
            preds = tree.predict(X_subset)
            tree_preds.append(preds)

        # транспонируем, чтобы получить матрицу, где каждая строка — предсказания всех деревьев для одного объекта
        tree_preds = np.array(tree_preds).T

        # голосование: выбираем наиболее частый класс - это и есть наш ответ
        final_preds = [Counter(row).most_common(1)[0][0] for row in tree_preds]
        return np.array(final_preds)


In [6]:
# обучение нашего леса
forest = RandomForest(n_estimators=50, max_depth=5, max_features=2)
# forest = RandomForest(n_estimators=10, max_depth=3) - никудышные параметры (accuracy 0.73)
forest.fit(X_train, y_train)

# предсказание
y_pred = forest.predict(X_test)

# оценка точности
acc = accuracy_score(y_test, y_pred)
print(f"Точность RandomForest: {acc:.2f}")

Точность RandomForest: 0.97


In [7]:
# модель рассчитывает уверенность в классе, а следующая учится на её ошибке, типа предыдущ. был на 0.8 уверена в классе А, а это класс Б. Тогда следующее дерево понизит уверенность
import numpy as np

class GradientBoostingClassifier:
    def __init__(self, n_estimators=10, learning_rate=0.1, max_depth=3, min_samples_split=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []
        self.init_pred = None

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        # преобразуем метки в {-1, 1}
        y_transformed = np.where(y == 1, 1, -1)
        self.init_pred = np.zeros(len(y))  # начальное предсказание — ноль

        for _ in range(self.n_estimators):
            # градиент логистической функции потерь
            residuals = y_transformed / (1 + np.exp(y_transformed * self.init_pred))

            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X, residuals)
            update = tree.predict(X)

            # обновляем предсказания
            self.init_pred += self.learning_rate * update
            self.trees.append(tree)

    def predict_proba(self, X):
        pred = np.zeros(X.shape[0])
        for tree in self.trees:
            pred += self.learning_rate * tree.predict(X)
        proba = self._sigmoid(pred)
        return np.vstack([1 - proba, proba]).T

    def predict(self, X):
        proba = self.predict_proba(X)
        return (proba[:, 1] >= 0.5).astype(int)


In [8]:
# обучаем градиентный бустинг
gb = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=2)
gb.fit(X_train, y_train)

# предсказания
y_pred = gb.predict(X_test)

# оценка точности
acc = accuracy_score(y_test, y_pred)
print(f"Точность Gradient Boosting: {acc:.2f}")


Точность Gradient Boosting: 0.97


In [9]:
# теперь всё то же самое для задачи регрессии
# вместо джинни - MSE; возвращаем среднее значение целевой переменной в листе вместо класса
class DecisionTreeRegressor:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def predict(self, X):
        return np.array([self._predict_sample(sample, self.tree) for sample in X])

    def _mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)

    def _best_split(self, X, y):
        best_mse = float('inf')
        best_index, best_threshold = None, None

        for feature_index in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask

                if len(y[left_mask]) < self.min_samples_split or len(y[right_mask]) < self.min_samples_split:
                    continue

                mse_left = self._mse(y[left_mask])
                mse_right = self._mse(y[right_mask])
                weighted_mse = (len(y[left_mask]) * mse_left + len(y[right_mask]) * mse_right) / len(y)

                if weighted_mse < best_mse:
                    best_mse = weighted_mse
                    best_index = feature_index
                    best_threshold = threshold

        return best_index, best_threshold

    def _build_tree(self, X, y, depth=0):
        if depth >= self.max_depth or len(y) < self.min_samples_split:
            return {'leaf': True, 'value': np.mean(y)}

        feature_index, threshold = self._best_split(X, y)
        if feature_index is None:
            return {'leaf': True, 'value': np.mean(y)}

        left_mask = X[:, feature_index] <= threshold
        right_mask = ~left_mask

        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {
            'leaf': False,
            'feature_index': feature_index,
            'threshold': threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    def _predict_sample(self, sample, tree):
        if tree['leaf']:
            return tree['value']
        if sample[tree['feature_index']] <= tree['threshold']:
            return self._predict_sample(sample, tree['left'])
        else:
            return self._predict_sample(sample, tree['right'])


In [10]:
# усредняем результаты голосования
class RandomForestRegressor:
    def __init__(self, n_estimators=10, max_depth=5, min_samples_split=2, max_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples, n_features = X.shape
        self.max_features = self.max_features or int(np.sqrt(n_features))

        for _ in range(self.n_estimators):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_sample = X[indices]
            y_sample = y[indices]

            feature_indices = np.random.choice(n_features, self.max_features, replace=False)
            X_subset = X_sample[:, feature_indices]

            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_subset, y_sample)
            self.trees.append((tree, feature_indices))

    def predict(self, X):
        predictions = []
        for tree, feature_indices in self.trees:
            X_subset = X[:, feature_indices]
            preds = tree.predict(X_subset)
            predictions.append(preds)
        return np.mean(predictions, axis=0)


In [11]:
# каждая последующая модель пытается предсказать то, что не хватило предыдущей
class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []
        self.init_pred = None

    def fit(self, X, y):
        self.init_pred = np.mean(y)
        pred = np.full(len(y), self.init_pred)

        for _ in range(self.n_estimators):
            residuals = y - pred
            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X, residuals)
            update = tree.predict(X)

            pred += self.learning_rate * update
            self.trees.append(tree)

    def predict(self, X):
        pred = np.full(X.shape[0], self.init_pred)
        for tree in self.trees:
            pred += self.learning_rate * tree.predict(X)
        return pred


In [12]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# 1. Генерация данных
X, y = make_regression(n_samples=200, n_features=5, noise=10, random_state=42)

# 2. Делим на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. Обучение моделей
dt = DecisionTreeRegressor(max_depth=5)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

rf = RandomForestRegressor(n_estimators=50, max_depth=5)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

gb = GradientBoostingRegressor(n_estimators=50, learning_rate=0.1, max_depth=3)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

# 4. Оценка качества
print(f"Decision Tree R²: {r2_score(y_test, y_pred_dt):.2f}")
print(f"Random Forest R²: {r2_score(y_test, y_pred_rf):.2f}")
print(f"Gradient Boosting R²: {r2_score(y_test, y_pred_gb):.2f}")


Decision Tree R²: 0.72
Random Forest R²: 0.53
Gradient Boosting R²: 0.89


In [13]:
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=2,
    max_features=X.shape[1]
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(f"Random Forest R² (обновлённый): {r2_score(y_test, y_pred_rf):.2f}")

Random Forest R² (обновлённый): 0.85
